In [1]:
import re
from datetime import datetime

In [2]:
with open('r1875-ivs-analysis-report.txt') as file:
    file_contents = file.read()
    print(file_contents)

       IVS Analysis Report for R1875 ($19JAN02XA) 
       This report is the official IVS analysis report that corresponds to 
       the database maintained by the IVS Data Centers for this session.  
   (Analyzed by David Gordon, Goddard Space Flight Center.
    Spoolfile source: interactive solve/nuSolve analysis.) 

Problems: KATH12M had DBBC problems. The ambiguity resolution was extremely
           difficult, almost but not quite impossible.
          YARRA12M has small clock breaks at ~02:00 and ~11:31 UTC.
          MATERA did not participate, brake problems. 
          NYALES20 missed ~7 hours due to strong winds. 

Parameterization comments: None. 

Other comments: None. 

-----------------------------------------

Session Statistics 

  Observations:     7426 scheduled 
                    4908 correlated (in database)
                    4713 recoverable (usable)
                    4653 used
  Session fit:        32.364 ps

-----------------------------------------

Stati

In [3]:
with open('r1875-ivs-analysis-report.txt') as file:
    contents = file.read()
    sections = contents.split('-----------------------------------------')

In [4]:
def problemFinder(text_section):
    stations = ['KATH12M', 'YARRA12M', 'HOBART12', 'HOBART26']
    problem_bool = []
    problem_string = []
    for ant in stations:
        regex = ant + '.*\n[\s]{11}.*|' + ant + '.*$'
        problem = re.findall(regex,text_section,re.MULTILINE)
        if len(problem) > 0:
            problem_bool.append(True)
            problem_string.append(problem[0].replace('\n', ""))
        else:
            problem_bool.append(False)
            problem_string.append('')
    return problem_bool, problem_string


In [5]:
problemFinder(sections[0])

([True, True, False, False],
 ['KATH12M had DBBC problems. The ambiguity resolution was extremely           difficult, almost but not quite impossible.',
  'YARRA12M has small clock breaks at ~02:00 and ~11:31 UTC.',
  '',
  ''])

In [6]:
def percent2decimal(percent_string):
    return float(percent_string.strip('%'))/100


def stationPerformance(text_section):
    stations = ['KATH12M', 'YARRA12M', 'HOBART12', 'HOBART26']
    station_performance = []
    for ant in stations:
        regex = ant + ".*"
        performance = re.findall(regex,text_section,re.MULTILINE)
        if len(performance) > 0:
            performance = percent2decimal(performance[0].split()[4])
            station_performance.append(performance)
        else:
            station_performance.append(None)
    
    return station_performance

In [7]:
stationPerformance(sections[2])

[0.645, 0.809, None, None]

In [8]:
def metaData(text_section):
    date = re.findall("(?<=\$).{7}",text_section,re.MULTILINE)
    date = datetime.strptime(date[0], '%y%b%d').strftime('%Y-%m-%d')
    exp_code = re.findall("(?<=Analysis Report for\s)(.*?(?=\s))",text_section,re.MULTILINE)
    analyser = re.findall("\S.*(?=\sAnalysis Report for\s)",text_section,re.MULTILINE)
    return exp_code[0], analyser[0], date
    # pretty sure this doesn't work post-2099, but like... you shouldn't be using this trash then... right?

In [9]:
metaData(sections[0])

('R1875', 'IVS', '2019-01-02')

In [10]:
with open('r1875-ivs-analysis-report.txt') as file:
    contents = file.read()
    sections = contents.split('-----------------------------------------')
    
meta = metaData(sections[0])
performance = stationPerformance(sections[2])
problems = problemFinder(sections[0])



In [11]:
import MySQLdb as mariadb
conn = mariadb.connect(user='root', passwd='', db='auscope_test')
cursor = conn.cursor()

In [12]:
sql_command = []
station_id = ['Ke', 'Yg', 'Hb', 'Ho']
for i in range(0, len(performance)):
    if performance[i] == None:
        break
    else:
        sql_station = "INSERT IGNORE INTO " + station_id[i] + " (ExpID, Performance, Date, Problem, Problem_String, Analyser) VALUES ('{}', {},'{}', {}, '{}' , '{}');".format(meta[0], performance[i], meta[2], problems[0][i], problems[1][i], meta[1])
        sql_command.append(sql_station)

# this snippet of code essentially checks if a performance metric exists (and therefore the station participated in the experiment)
# If this is true, it generates an SQL command string for that station. 

In [13]:
sql_command

["INSERT IGNORE INTO Ke (ExpID, Performance, Date, Problem, Problem_String, Analyser) VALUES ('R1875', 0.645,'2019-01-02', True, 'KATH12M had DBBC problems. The ambiguity resolution was extremely           difficult, almost but not quite impossible.' , 'IVS');",
 "INSERT IGNORE INTO Yg (ExpID, Performance, Date, Problem, Problem_String, Analyser) VALUES ('R1875', 0.809,'2019-01-02', True, 'YARRA12M has small clock breaks at ~02:00 and ~11:31 UTC.' , 'IVS');"]

In [14]:
for i in range(0,len(sql_command)):
    cursor.execute(sql_command[i])

conn.commit()
conn.close() 

In [15]:
conn = mariadb.connect(user='root', passwd='', db='auscope_test')
cursor = conn.cursor()
query = "SELECT ExpID FROM Ke"
cursor.execute(query)
result_list = [item for sublist in cursor.fetchall() for item in sublist]
print(result_list)

['R1875', 'R1876temp']


In [23]:
station_id = ['Ke', 'Yg', 'Hb', 'Ho']
conn = mariadb.connect(user='root', passwd='')
cursor = conn.cursor()
query = "CREATE OR REPLACE DATABASE auscope_test;"
cursor.execute(query)
conn.commit()
query = "USE auscope_test"
cursor.execute(query)
conn.commit()

for ant in station_id:
    query = "CREATE TABLE IF NOT EXISTS "+ ant + " (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, Performance decimal(4,3) NOT NULL, Date DATETIME, Pos_X decimal(14,2), Pos_Y decimal(14,2), Pos_Z decimal(14,2), Problem BIT(1), Problem_String VARCHAR(100), Analyser VARCHAR(10) NOT NULL);" 
    cursor.execute(query)
    conn.commit()
    
conn.close()
    

In [25]:
station_id = ['Ke', 'Yg', 'Hb', 'Ho']
conn = mariadb.connect(user='root', passwd='')
cursor = conn.cursor()
query = "CREATE DATABASE IF NOT EXISTS auscope_test;"
cursor.execute(query)
conn.commit()
query = "USE auscope_test"
cursor.execute(query)
conn.commit()
for ant in station_id:
    query = "CREATE TABLE IF NOT EXISTS "+ ant + " (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, Performance decimal(4,3) NOT NULL, Date DATETIME, Pos_X decimal(14,2), Pos_Y decimal(14,2), Pos_Z decimal(14,2), Problem BIT(1), Problem_String VARCHAR(100), Analyser VARCHAR(10) NOT NULL);" 
    cursor.execute(query)
    conn.commit()
    
conn.close()